# Data exploration - sorting data

AAA

**Version log**

- Created by Robert Nyqvist 2022-12-13

## I. Libs & imports

### A. Notebook imports

In [1]:
# Standard imports
import os
import sys
import warnings
from jinja2 import Template
from pathlib import Path
from typing import Optional

# 3rd party imports
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Own imports
from log_sorting.config import load_config
from log_sorting.constants import (
    CONFIG_DIR,
    CONFIG_FILES,
    CROSS_JOIN_COL,
    CSV_EXTENSION,
    DATA_DIR,
    DATE_COL,
    DESIRED_COL,
    DIAMETER_COL,
    DIAMETER_GROUP_COL,
    LENGTH_COL,
    LOT_ID_COL,
    SPECIES_COL,
    SQL_DIR,
)
from log_sorting.db_utils import load_sql_query, read_to_pandas
from log_sorting.functions import load_and_preprocess_red_sheet

# Print out version numbers
print('Python       ' + sys.version[0:5])
print('Numpy        ' + np.__version__)
print('Pandas       ' + pd.__version__)
print('Matplotlib   ' + mpl.__version__)

# Supress warnings
warnings.filterwarnings('ignore')

Python       3.8.1
Numpy        1.21.6
Pandas       1.3.5
Matplotlib   3.6.2


## II. Run

In [17]:
# Load config
config = load_config(CONFIG_FILES, CONFIG_DIR)

# Load red sheet

red_sheet, query_config = load_and_preprocess_red_sheet(
    file_name=config.file_names.red_sheet,
    file_dir=DATA_DIR,
)

# Make sql query
q = load_sql_query("log-sorting-daily", SQL_DIR)
prepped_query = Template(q).render(
    species=config.sql.species,
    start_date=config.sql.start_date,
    end_date=config.sql.end_date,
    diameter_groups=query_config,
)

# Read from database
sql_df = read_to_pandas(config.database.mapptvr, prepped_query)

In [22]:
# Create base data: All combinations of
#   - 'species'
#   - 'date'
#   - 'lot_id'
#   - 'diameter_group'
#   - 'length'
base_df = sql_df[[SPECIES_COL, DATE_COL, LOT_ID_COL]].drop_duplicates()
base_df[CROSS_JOIN_COL] = 0
diameter_groups_lengths = sql_df[[DIAMETER_GROUP_COL, LENGTH_COL]].drop_duplicates()
diameter_groups_lengths[CROSS_JOIN_COL] = 0
sort_cols = [
    SPECIES_COL,
    DATE_COL,
    LOT_ID_COL,
    DIAMETER_GROUP_COL,
    LENGTH_COL,
]
base_df = (
    base_df
    .merge(
        right=diameter_groups_lengths,
        on=CROSS_JOIN_COL,
        how="outer",
        validate="m:m",
    )
    .sort_values(sort_cols)
    .drop(columns=[CROSS_JOIN_COL])
)
# Merge with results & red sheet data
df = (
    base_df
    .merge(
        right=sql_df,
        on=[
            SPECIES_COL,
            DATE_COL,
            LOT_ID_COL,
            DIAMETER_GROUP_COL,
            LENGTH_COL,
        ],
        how="left",
        validate="1:1",
    )
    .astype({LENGTH_COL: float})
    .merge(
        right=red_sheet,
        on=[DIAMETER_GROUP_COL, LENGTH_COL],
        how="inner",
        validate="m:1",
    )
    .fillna(0)
)

In [23]:
print("sql_df: ", sql_df.shape)
print("base_df: ", base_df.shape)
print("temp_df: ", temp_df.shape)
print("df: ", df.shape)

sql_df:  (75023, 7)
base_df:  (133488, 5)
temp_df:  (133488, 7)
df:  (133488, 8)
